<a href="https://colab.research.google.com/github/JRopes/CrystalEnergyPredictionWithInvariants/blob/main/AMD_GaussianProcess_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SETUP**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab_Notebooks/Dissertation/Prediction_Prototyping') 

**IMPORTING LIBRARY DEPENDENCIES**

In [ ]:
import numpy as np
import pickle
import math
import pandas as pd

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RationalQuadratic, RBF, ConstantKernel, Matern, ExpSineSquared

**IMPORTING DATA**

In [ ]:
feature_dir_path = '/content/drive/MyDrive/Colab_Notebooks/Dissertation/Data/AMDs_T2_1000.csv'

In [ ]:
RAW_DATA = pd.read_csv(feature_dir_path)
data = RAW_DATA.to_numpy()

In [ ]:
features = data[:,4:]
labels = data[:,1]

In [ ]:
pickle.dump(features, open("amd_feature_data_forest.p", "wb"))

pickle.dump(labels, open("amd_label_data_forest.p","wb"))

In [ ]:
def data():
  feature_data = pickle.load(open("amd_feature_data_forest.p","rb"))
  label_data = pickle.load(open("amd_label_data_forest.p","rb"))

  feature_data = np.nan_to_num(feature_data)

  feature_data = feature_data[:,:100]

  ## MinMax Scaler
  feature_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
  X_scaled = feature_scaler.fit_transform(feature_data)
  
  label_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
  y_scaled = label_scaler.fit_transform(label_data.reshape(-1,1))

  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, shuffle=True)

  return X_train, y_train, X_test, y_test, label_scaler

**ARCHITECTURE**

In [ ]:
X_train, y_train, X_test, y_test, label_scaler = data()

In [ ]:
kernel = RationalQuadratic()

In [ ]:
gpr = GaussianProcessRegressor(kernel=kernel)

**TRAINING**

In [ ]:
gpr.fit(X_train, y_train)

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
                         kernel=RationalQuadratic(alpha=1, length_scale=1),
                         n_restarts_optimizer=0, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=None)

**EVALUATING**

In [ ]:
mean_predictions, std_predictions = gpr.predict(X_test, return_std=True)

In [ ]:
std_predictions = std_predictions.reshape(-1,1)

In [ ]:
scaler = np.divide(std_predictions, mean_predictions)

In [ ]:
mean_predictions = label_scaler.inverse_transform(mean_predictions)
std_predictions = np.multiply(scaler,mean_predictions)
y_test = label_scaler.inverse_transform(y_test)

In [ ]:
average_loss = 0
average_loss_percentage = 0
average_loss_percentage_rel_range = 0
counter = 0
rms = 0

error_ranges = np.array((0,0,0,0,0,0))

max_value = -999999.99
min_value = 999999.99

for label in y_test:
    if(label > max_value):
        max_value = label
        
    if(label < min_value):
        min_value = label
        
label_range = abs(max_value - min_value)


for i,prediction in enumerate(mean_predictions):
    percentage_difference = abs((abs(prediction - y_test[i]) / y_test[i]) * 100)
    percentage_difference2 = abs((abs(prediction - y_test[i]) / label_range) * 100)
    loss = abs(prediction - y_test[i])
    average_loss += loss

    rms += loss**2

    if(loss <= 1.0):
      error_ranges[0] += 1
    elif(loss <= 2.0):
      error_ranges[1] += 1
    elif(loss <= 4.0):
      error_ranges[2] += 1
    elif(loss <= 8.0):
      error_ranges[3] += 1
    elif(loss <= 10.0):
      error_ranges[4] += 1
    else:
      error_ranges[5] += 1

    average_loss_percentage += percentage_difference
    average_loss_percentage_rel_range += percentage_difference2
    counter += 1

rms = math.sqrt(rms / counter)

print()
print("SUMMARY:")
print()
print("Root Mean Squared Error: " + str(rms))
print("Mean Absolute Error: " + str(average_loss / counter))
print("Mean Absolute Percentage Error: " + str(average_loss_percentage / counter) + "%")
print("Mean Absolute Percentage Error relative to Label Range: " + str(average_loss_percentage_rel_range / counter) + "%")
print("Accuracy: " + str(100 - (average_loss_percentage / counter)) + "%")
print()
print("BREAKDOWN:")
print("   Error <= 1.0 kJ/mol: " + str(error_ranges[0]) + " or " + str((error_ranges[0] / counter) * 100) + "% of Test Set")
print("   Error <= 2.0 kJ/mol: " + str(error_ranges[1]) + " or " + str((error_ranges[1] / counter) * 100) + "% of Test Set")
print("   Error <= 4.0 kJ/mol: " + str(error_ranges[2]) + " or " + str((error_ranges[2] / counter) * 100) + "% of Test Set")
print("   Error <= 8.0 kJ/mol: " + str(error_ranges[3]) + " or " + str((error_ranges[3] / counter) * 100) + "% of Test Set")
print("   Error <= 10.0.0 kJ/mol: " + str(error_ranges[4]) + " or " + str((error_ranges[4] / counter) * 100) + "% of Test Set")
print("   Error > 10.0 kJ/mol: " + str(error_ranges[5]) + " or " + str((error_ranges[5] / counter) * 100) + "% of Test Set")
print("----------------------------------------------------------------------------------------------")


SUMMARY:

Root Mean Squared Error: 5.9887452538781885
Mean Absolute Error: [4.56179842]
Mean Absolute Percentage Error: [3.25551282]%
Mean Absolute Percentage Error relative to Label Range: [4.56242348]%
Accuracy: [96.74448718]%

BREAKDOWN:
   Error <= 1.0 kJ/mol: 82 or 14.43661971830986% of Test Set
   Error <= 2.0 kJ/mol: 85 or 14.964788732394366% of Test Set
   Error <= 4.0 kJ/mol: 149 or 26.23239436619718% of Test Set
   Error <= 8.0 kJ/mol: 160 or 28.169014084507044% of Test Set
   Error <= 10.0.0 kJ/mol: 38 or 6.690140845070422% of Test Set
   Error > 10.0 kJ/mol: 54 or 9.507042253521126% of Test Set
----------------------------------------------------------------------------------------------


**Prediction with Uncertainty**

In [ ]:
instance = 23

In [ ]:
print("Prediction: " + str(mean_predictions[instance,0]) + " +- " + str(-1.96 * std_predictions[instance,0]) + " with 95% Confidence || True Label: " + str(y_test[instance]))

Prediction: -158.0458872154617 +- 27.584631893126225 with 95% Confidence || True Label: [-153.3745]
